In [44]:
# После установки появится сообщение о опасности (WARNING).
# Нажмите кнопку RESATART RUNTIME, теперь на нашей удалённой машине используется нужная версия TF.
# Убедимся в этом, выполнив вывод версии TF.
import tensorflow
print(tensorflow.__version__)

2.12.1


In [45]:
# импортируем необходимые для работы библиотеки и функции

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import random
import pickle
import cv2
import os

#библиотека, используемая для построения графиков обучения нейронной сети
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("Agg")


print("Done")

Done


In [46]:
import pandas as pd

csv_path = "C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train.csv" # укажите путь к файлу с аннотациями
image_path = "C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train" # укажите путь к папке с изображениями для тренировки

table = pd.read_csv(csv_path, delimiter=';')

row_num = len(table.axes[0])  # число строк (первая строка с заголовками столбцов не учитывается)
col_num = len(table.axes[1])  # учитываются только столбцы оригинальной таблицы

table

,Unnamed: 0,path,label
0,13692,0047b3de716d49879b791fdb6a6158dd.jpg,alpha
1,17894,0277e04983b142edb6c13a6dd3d57c2b.jpg,alpha
2,18226,8c54aed60df047a7a55150b9a38876b2.jpg,alpha
3,18037,a72c3d8bf8c54237bbfc64ce56dcf1a6.jpg,alpha
4,20331,52e80a34145d40c3aa22de1bd925a0a7.jpg,-
...,...,...,...
6346,17771,cce5552565194e978deb3c88f93e9a31.jpg,alpha
6347,17363,d60f071ea4d8447db5dbacc3c6ab2fdb.jpg,alpha
6348,19146,708031f24b234c3ba3d47c4992a46afc.jpg,beta
6349,14390,7fa68d0a126f4e99bbb46ea7152c47e3.jpg,beta


In [47]:
# пустые списки для изображений и аннотаций (меток) к ним
data = []
labels = []

for row in table.itertuples(): # перебираем строки
  # к элементу строки можно обращаться по индексу
  # индекс 0 - соответствует номеру строки, автоматическми добавляемому pandas
  image_type = row[3] # тип изображения записан в четвёртом столбце
  if image_type == "alpha" or image_type == "beta": # какие изображения мы хотим выбрать?
    image = cv2.imread(image_path + '/' + row[2]) #  читаем файл с именем, записанным в третьем столбце
    print(image_path + row[2])
    if image is None:
      print('FFFFUUUUUUUUCK!!!')
      pass

    image = cv2.resize(image, (32, 32))
    data.append(image.flatten()) # flatten - выдаёт все элементы в формате одномерного массива
    if image_type == "alpha":
      label = [1, 0]
    elif image_type == "beta":
      label = [0, 1]
    labels.append(label)

  # небольшой счётчик, чтобы был ясен прогресс выполнения этого блока кода
  if row[0] % 100 == 0:
    print(str(row[0]) + " из " + str(row_num))

data = np.array(data, dtype="float") / 255 # создаём из списка массив и нормируем изображения на 1
labels = np.array(labels) # создаём из списка массив numpy

# сохраняем массив c изображениями
with open("C:/Users/jibra/Documents/OpenCV/image.txt", 'wb') as f:
  pickle.dump(data, f)
print("Data seved")

# сохраняем список с лэйблами - метками
with open("C:/Users/jibra/Documents/OpenCV/list.txt", 'wb') as f:
  pickle.dump(labels, f)
print("Labels seved")

# Если вы один раз выполнили этот блок кода, у вас появятся файлы ".picle"
# В них сохранены массивы с изображениями и соответствующими метками.
# Когда вы будете работать с датасетом в следующий раз,
# не придётся снова работать csv файлом, читать изоражения с диска и формировать для них метки.
# Достаточно загрузить данные из файлов ".picle", в следующем блоке кода.


C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train0047b3de716d49879b791fdb6a6158dd.jpg
0 из 6351
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train0277e04983b142edb6c13a6dd3d57c2b.jpg
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train8c54aed60df047a7a55150b9a38876b2.jpg
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_traina72c3d8bf8c54237bbfc64ce56dcf1a6.jpg
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_trainc680de7c368f44b1a40e37a36c35517e.jpg
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_trainbaaf5dbe05d34121b0f663fbd8ad8a16.jpg
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train953a609231a547f59e0c0be4ce07a640.jpg
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train47c83f29735d4526beec2d4a85957308.jpg
C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_traina914ec0c63484d1db23ee8b9cb5e3fea.jpg
C:/Users/jibra/Dow

In [48]:
# загружаем массив с изображениями с диска
with open("C:/Users/jibra/Documents/OpenCV/image.txt", 'rb') as f:
  data = pickle.load(f)
print("Data loaded")

# загружаем список с лейблами-метками
with open("C:/Users/jibra/Documents/OpenCV/list.txt", 'rb') as f:
  labels = pickle.load(f)
print("Labels loaded")

# делим подготовленные данные на тренировочную и валидационную часть
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.15,  # доля изображений для проверки
                                                  random_state=42)

print("Dataset prepared")

Data loaded
Labels loaded
Dataset prepared


In [49]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation
# from tensorflow.keras.layers import Dense, Dropout, Flatten
# from tensorflow.keras.optimizers import SGD, Adam
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# мы импортировали всё необходимое в блоке кода выше

model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation='sigmoid'))
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))

INIT_LR = 0.001
opt = SGD(learning_rate=INIT_LR)
#opt = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)

model.compile(loss="binary_crossentropy",optimizer=opt,
              metrics=["accuracy"])  #categorial_crosentropy

print ("Model compiled")
model.summary()

Model compiled
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1024)              3146752   
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 3,672,578
Trainable params: 3,672,578
Non-trainable params: 0
_________________________________________________________________


In [50]:
# тренируем нейронную сеть
# меняйте гиперпараметры в блоке кода выше и здесь, чтобы получить точность выше
EPOCHS = 7

# checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/NSK_Lesson5/ConvNN_{epoch:2d}_{val_accuracy:.3f}.h5', verbose=1, period=10)
checkpointer = ModelCheckpoint(filepath='C:/Users/jibra/Documents/OpenCV/AI.txt', verbose=1, save_best_only=True)
# checkpointer = ModelCheckpoint(filepath='/content/drive/MyDrive/NSK_Lesson5/point.h5', verbose=1)

H = model.fit(trainX, trainY, validation_data=(testX, testY),
              epochs=EPOCHS, batch_size=32,
              shuffle=True,
              callbacks=[checkpointer])
             #callbacks=[checkpointer1, checkpointer2, checkpointer3]) # если используете несколько разных call-back функций
             # https://keras.io/api/callbacks/

print("Model trained")

Epoch 1/7
120/122 [============================>.] - ETA: 0s - loss: 0.6678 - accuracy: 0.6182
Epoch 1: val_loss improved from inf to 0.64153, saving model to C:/Users/jibra/Documents/OpenCV\AI.txt


INFO:tensorflow:Assets written to: C:/Users/jibra/Documents/OpenCV\AI.txt\assets


INFO:tensorflow:Assets written to: C:/Users/jibra/Documents/OpenCV\AI.txt\assets


122/122 [==============================] - 2s 16ms/step - loss: 0.6673 - accuracy: 0.6189 - val_loss: 0.6415 - val_accuracy: 0.6447
Epoch 2/7
121/122 [============================>.] - ETA: 0s - loss: 0.6431 - accuracy: 0.6229
Epoch 2: val_loss improved from 0.64153 to 0.62358, saving model to C:/Users/jibra/Documents/OpenCV\AI.txt


INFO:tensorflow:Assets written to: C:/Users/jibra/Documents/OpenCV\AI.txt\assets


...

INFO:tensorflow:Assets written to: C:/Users/jibra/Documents/OpenCV\AI.txt\assets


122/122 [==============================] - 2s 15ms/step - loss: 0.5751 - accuracy: 0.6429 - val_loss: 0.5564 - val_accuracy: 0.6959
Epoch 7/7
121/122 [============================>.] - ETA: 0s - loss: 0.5576 - accuracy: 0.6805
Epoch 7: val_loss improved from 0.55641 to 0.53750, saving model to C:/Users/jibra/Documents/OpenCV\AI.txt


INFO:tensorflow:Assets written to: C:/Users/jibra/Documents/OpenCV\AI.txt\assets


INFO:tensorflow:Assets written to: C:/Users/jibra/Documents/OpenCV\AI.txt\assets


122/122 [==============================] - 2s 15ms/step - loss: 0.5576 - accuracy: 0.6806 - val_loss: 0.5375 - val_accuracy: 0.6857
Model trained


In [51]:
# Этот блок кода выводит статистику обучения нейронной сети


predictions = model.predict (testX, batch_size = 32)
print(predictions)

print(classification_report(testY.argmax(axis=1),
                            predictions.argmax(axis=1),
                            target_names=["alpha", "beta"])) # не забудьте поставить тут правильные имена классов

N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="vall_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Results")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy/")
plt.legend()
plt.show()

# строки ниже сохраняли бы последнее состояние модели и график обучения
plt.savefig("C:/Users/jibra/Documents/OpenCV/Loss.png")
# model.save("/content/drive/MyDrive/GoogleColab/Lesson_4/EasyNet.h5")

import cv2
img = cv2.imread("C:/Users/jibra/Documents/OpenCV/Loss.png")
cv2.imshow("loss", img)


print("End")

22/22 [==============================] - 0s 5ms/step
[[0.6764275  0.3235725 ]
 [0.8702132  0.12978677]
 [0.81448156 0.18551841]
 ...
 [0.6651661  0.33483395]
 [0.88367337 0.11632662]
 [0.8086894  0.19131061]]
              precision    recall  f1-score   support

       alpha       0.67      1.00      0.80       441
        beta       1.00      0.12      0.21       243

    accuracy                           0.69       684
   macro avg       0.84      0.56      0.51       684
weighted avg       0.79      0.69      0.59       684

End


C:\Users\jibra\AppData\Local\Temp\ipykernel_3632\3130253939.py:22: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [52]:
# Пример того, загрузить модель из файла
model = tf.keras.models.load_model("C:/Users/jibra/Documents/OpenCV/AI.txt")
print("Model loaded")

Model loaded


In [53]:
# Пример того, как можно подать на вход сети одно единственное изображение и получить ответ
image_path = "C:/Users/jibra/Downloads/handwritten_short/handwritten_short/data_train/0a0b46be7a4143b0a7f37091eb0985dd.jpg"
image = cv2.imread(image_path)
cv2.imshow("AA", image)
image1 = cv2.resize(image, (32, 32)) / 255
image1 = image1.flatten()
image1 = np.expand_dims(image1, axis=0) # сеть принимает на вход изображение с добавленным измерением, посмотрите как меняется форма массива после этой команды
pred = model.predict(image1)
print(pred)
print(pred.shape) #форма выходного массива, он двумерный

if pred[0, 0] >= pred[0, 1]: # выясняем какой нейрон выдал большее значение
    print("alpha")
else:
    print("beta")

1/1 [==============================] - 0s 33ms/step
[[0.79375726 0.20624268]]
(1, 2)
alpha
